# Sensitivity studies using COSIPY

In this tutorial we will go through how to do a sensitivity study of the surface mass balance of a glacier using COSIPY. For a sensitivity study we want to check how much changing a constant affects the calculation of a certain variables, i.e. how sensitive it is. We can do this on essentially any variable, but in our case we will focus on the surface mass balance of a glacier.

Just as in [First steps in COSIPY](first_steps.ipynb) we begin with importing some useful libraries and the cosipy functions we need.

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
import sys
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')

In [ ]:
from cosipy.utils import edu_utils
# cfg gives us the NAMELIST
import cfg
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

## Choosing the constant to investigate

Before we can continue with setting up the simulations, we have to choose the constant we want to change. As you might remember, all the constants that are used by COSIPY are stored in the NAMELIST. Let's take a look at it

In [ ]:
cfg.NAMELIST

This contains a lot of different things and can be a bit overwhelming at first. It contains everything from the configuration of where to store files to constants used in some of the submodules used for example calculating the densification of snow.

If we plan to look at the surface mass balance in winter, the albedo of fresh  snow is a good guess of a variable that might have an impact on the calculations.

In [ ]:
cfg.NAMELIST['albedo_fresh_snow']

Let's say we want to change this 10% in either direction. Since the model is fetching the constants from `NAMELIST`, the plan is to change the albedo of fresh snow in `NAMELIST` before we use it to intialize the model.

### Initializing the namelists and data files

We begin by initialising three instances of the namelist. We then keep one as default while we change the albedo of fresh snow in the two others

In [ ]:
# We create a copy of the namelist so we don't change the original.
# Default
NAMELIST_def = cfg.NAMELIST.copy()
# Up 10%
NAMELIST_up = cfg.NAMELIST.copy()
# We overwrite the default value. We multiply it from cfg since we can then
# run the cell multiple times without performing the multiplication multiple
# times.
NAMELIST_up['albedo_fresh_snow'] = cfg.NAMELIST['albedo_fresh_snow'] * 1.1
# Down 10%
NAMELIST_dn =cfg.NAMELIST.copy()
NAMELIST_dn['albedo_fresh_snow'] = cfg.NAMELIST['albedo_fresh_snow'] * 0.9

Did it work?

In [ ]:
print(f"NAMELIST_dn: {NAMELIST_dn['albedo_fresh_snow']:.3f}\nNAMELIST_def: \
{NAMELIST_def['albedo_fresh_snow']:.3f}\nNAMELIST_up: \
{NAMELIST_up['albedo_fresh_snow']:.3f}")

<div class="alert alert-warning">
    <b>Question: Can you figure out what to change in the cells above to alter the albedo by 5% instead of 10%?</b>
</div>

We can now go on to creating the files used for storing the data and results of the simulation. Just as with the namlist we need one set of instances for each experiment we want to run, in this case three. 

In [ ]:
# Default
IO_def, DATA_def, RESULTS_def = edu_utils.create_IO(NAMELIST_def)
# Up
IO_up, DATA_up, RESULTS_up = edu_utils.create_IO(NAMELIST_up)
# Down
IO_dn, DATA_dn, RESULTS_dn = edu_utils.create_IO(NAMELIST_dn)

## Running multiple simulations

We have now prepared the namelists and data/results files for the three different runs we want to do. The runs can initilaized as it was done in the [First steps in COSIPY](first_steps.ipynb) notebook, and changing the namelist and data/results variables manually for each run. But to save us writing the same code multiple times, and to give some inspiration how one might go about running a higher number of simulations, we will now do it in a loop. First we have to put all we need into a list

In [ ]:
# List of lists with our experiments
exp_list = [[NAMELIST_def, DATA_def, RESULTS_def, IO_def],
            [NAMELIST_dn, DATA_dn, RESULTS_dn, IO_dn],
            [NAMELIST_up, DATA_up, RESULTS_up, IO_up]
           ]

We can index this list to get the results of experiment 2 as `exp_list[1][2]`.

<div class="alert alert-info">
    <b>Remember that python is zero-indexed.</b>
</div>

Let's setup the loop and run the simulations. This will take some time.

In [ ]:
for exp in exp_list:
    # Call run_model once for each experiment
    edu_utils.run_model(exp[1], exp[3], exp[0], exp[2])

## A look at the results

When the simulations has finished running it is time to take a look at the results. Since we're keeping the results in `exp_list` we can again leverage the power of loops to save us writing some code

In [ ]:
labels = ['Default', '-10%', '+10%']
fig, ax = plt.subplots(figsize=(12, 5))
for exp, label in zip(exp_list, labels):
    # Get the data and plot it RESULTS are kept at the third spot, index 2
    exp[2].surfMB.plot(ax=ax, label=label)
plt.legend(); 

<div class="alert alert-warning">
<details>
    <summary>
    <b>Question: Can you explain why the surface mass balance looks like it does?</b> <i>Click me for an explanation!</i></summary>
<p>We are looking at three different surface mass balance series calculated with three different values for the albedo of fresh snow. The regular dips in the surface mass balance is the diurnal cycle, the glacier lose mass during the day while it remains constant during the night. When the albedo is increased the surface of the glacier will reflect more of the incoming solar radiation during the day. This leads to a decrease in the energy available for melt, thus reducing the melt and resulting in a less negative surface mass balance.
<br><br>
On the other hand, when the albedo is decreased the reflectivity of the glacier is reduced. This increases the energy available for melt which is why the surface mass balance is more negative.</p>
</details>
</div>

*Here you can write your answer (double click to open the cell)* 


## Another sensitivity study

Try to come up with another variable to use for a sensitivity study. The loop for running the simulation is provided below, but you have to initialise the namelist(s), data and result files and the list containing your experiments.

In [ ]:
# Pick a variable to nudge


In [ ]:
# Initialise the namelists and datafiles


In [ ]:
# Pack it all into a list as above


<div class="alert alert-info">
    Are you ready to run the simulation?
</div>

In [ ]:
for exp in exp_list:
    # Call run_model once for each experiment
    edu_utils.run_model(exp[1], exp[3], exp[0], exp[2])

Try plotting the results of your study!

In [ ]:
# Do some plotting here.


## Next steps
[Back to overview](welcome.ipynb)

[Temperature bias experiments](temp_sensitivity.ipynb)
